## 百度地图API-逆地理编码 （使用经纬度得到完整地址, 需要申请api秘钥AK）

http://lbsyun.baidu.com/index.php?title=webapi/guide/webservice-geocoding-abroad

### 结果示例：


{'status': 0, 'result': {'location': {'lng': 116.46442926383153, 'lat': 39.902039045066715}, 'formatted_address': '北京市朝阳区惠力东路', 'business': '双井,广渠门,劲松', 'addressComponent': {'country': '中国', 'country_code': 0, 'country_code_iso': 'CHN', 'country_code_iso2': 'CN', 'province': '北京市', 'city': '北京市', 'city_level': 2, 'district': '朝阳区', 'town': '', 'adcode': '110105', 'street': '惠力东路', 'street_number': '', 'direction': '', 'distance': ''}, 'pois': [], 'roads': [], 'poiRegions': [], 'sematic_description': '', 'cityCode': 131}}

### 字段介绍：

**status**: 一般为0， 表示正确返回，若返回值为其他参数则看上面的链接。

**result**：完整的结果，格式为Dictionary

**address**: 完整地址，对应['formatted_address']

**zipcode**：邮政编码，对应['addressComponent']['adcode']

**country**： 国家， 对应['addressComponent']['country']

**province**： 省份， 对应['addressComponent']['province']

**city**： 城市， 对应['addressComponent']['city']

**district**：行政区， 对应 ['addressComponent']['district']

**street**：街道， 对应 ['addressComponent']['street']

**street_number**：门牌号，对应['addressComponent']['street_number']

**business**：半径1000米内的POI，最多返回三个，对应['business']

In [ ]:
import pandas as pd
import numpy as np
import requests
import json

In [ ]:
#use baidumap API
url = "http://api.map.baidu.com/reverse_geocoding/v3/?ak=您的AK"

# 构建需要被检索的经纬度列表
com_lon=np.array(df['longitude'][:])
com_lat=np.array(df['latitude'][:])

# 结果存储器
result = [] # result = list(np.arange(0, 100)) 如果程序意外终止，只保存了部分资料然后需要重新运行时，使用左边的代码，手动修改数字
address = []
zipcode = []
country = []
province = []
city = []
district = []
street = []
street_number = []
business= []

print('start getting address data')

m = 0  #可能会中途中断连接，但结果已经储存在上面的[]中了，改 m,n值即可继续爬取
n = df.shape[0]

for i in range (m , n):
    lon=com_lon[i]
    lat=com_lat[i]
    location = "&location="+str(lat)+","+str(lon)
    URL = url+location
    response = requests.get(URL)
    answer = response.json()
    #如果API服务正常，则status == 0
    try:
        if answer['status'] == 0:
            result.insert(i, answer)
            address.insert(i, answer['result']['formatted_address'])
            zipcode.insert(i, answer['result']['addressComponent']['adcode'])
            country.insert(i, answer['result']['addressComponent']['country'])
            province.insert(i, answer['result']['addressComponent']['province'])
            city.insert(i,answer['result']['addressComponent']['city'])
            district.insert(i, answer['result']['addressComponent']['district'])
            street.insert (i, answer['result']['addressComponent']['street'])
            street_number.insert (i, answer['result']['addressComponent']['street_number'])
            business.insert(i, answer['result']['business'])
            print('done for', i, len(result),((i+1)== len(result) == len(address) == len(zipcode) == len(country) == len(province)
                                              == len(city) == len(district) == len(street) == len(street_number) == len(business)))
            time.sleep(0.3) #加入sleep函数，如果获取速度过快，很容易被封(connection error)， 建议0.3S以上
        else: 
            result.insert(i, 0)
            address.insert(i, 0)
            zipcode.insert(i, 0)
            country.insert(i, 0)
            province.insert(i, 0)
            city.insert(i,0)
            district.insert(i, 0)
            street.insert (i, 0)
            street_number.insert (i, 0)
            business.insert(i, 0)
    except:
        continue #可能会碰上OS Error, Connection Error等问题
        
# 将解析好的内容变成一个新的dataframe
data = pd.DataFrame({'result':result,'address':address,
                    'zipcode':zipcode, 'country': country,
                    'province': province, 'city': city,
                    'district': district, 'street': street,
                    'street_number': street_number,
                    'business': business})

print('finish getting address data')
print(data.shape)
    
# 在原始的dataframe里面新建对应的column    
#df['result'] = ""
#df['address'] = ""
#df['zipcode'] = ""
#df['country'] = ""
#df['province'] = ""
#df['city'] = ""
#df['district'] = ""
#df['street'] = ""
#df['street_number'] = ""
#df['business']= ""

print('start exporting data')

# 输出
for i in range(m, n):
    df['result'][i] = data['result'][i]
    df['address'][i] = data['address'][i]
    df['zipcode'][i] = data['zipcode'][i]
    df['country'][i] = data['country'][i]
    df['province'][i] =  data['province'][i]
    df['city'][i] = data['city'][i]
    df['district'][i] = data['district'][i]
    df['street'][i] = data['street'][i]
    df['street_number'][i] = data['street_number'][i]
    df['business'][i] = data['business'][i]

# 保存为csv
df.to_csv('listing_with_address.csv', index = False, encoding = 'utf-8-sig')
print('finish')
